In [3]:
import os

import earthpy as et
import geopandas as gpd
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import box
import warnings
from utils.crop import save_drapp_shapefile
from utils.crop import save_drapp_tiles
from utils.crop import plot_aoi_bbox_drapp
from utils.crop import check_contiguity

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME, 'treebeard')
drapp_dir = os.path.join(data_dir, 'drapp_tile_scheme_2020')
drapp_tiles_dir = os.path.join(data_dir, 'drapp_tiles')
aoi_shapefiles = [
    'assets/areas/immediate_project/Zumwinkel_property.shp',
    'assets/areas/planning/Potential Project Area_1.shp',
    'assets/areas/planning/Potential Project Area.shp',
    'assets/areas/planning/ProjectOverview.shp'
]

In [4]:
study_area = aoi_shapefiles[1]
aoi_gdf = gpd.read_file(study_area)
# Add checks for contiguous areas? Throw error if not?
# if not check_contiguity(aoi_gdf):
#     raise ValueError('The area of interest is not contiguous')
# Is checking for MultiPolygon the right approach?
if len(aoi_gdf) > 1:
    if 'Potential Project Area_1.shp' in study_area:
        aoi_gdf = aoi_gdf.head(2)
    elif 'Potential Project Area.shp' in study_area:
        # the last item
        aoi_gdf = aoi_gdf.tail(1)
        # aoi_gdf = aoi_gdf[2:3]
    aoi_gdf = aoi_gdf.dissolve()
if not aoi_gdf.crs or aoi_gdf.crs.to_epsg() != 4326:
    aoi_gdf.to_crs(epsg=4326, inplace=True)

In [6]:
# Find bounding box of AOI
bbox_aoi_gdf = gpd.GeoDataFrame(
    geometry=aoi_gdf.bounds.apply(
        lambda row: box(
            row['minx'], row['miny'], 
            row['maxx'], row['maxy']), axis=1))
bbox_aoi_gdf.crs = aoi_gdf.crs

# Download DRAPP shapefile
drapp_shapefile = save_drapp_shapefile(drapp_dir)
# Load DRAPP shapefile
drapp_gdf = gpd.read_file(drapp_shapefile)
drapp_gdf = drapp_gdf.to_crs(aoi_gdf.crs) # EPSG:6428 -> EPSG:4326
# Spatial join to find intersecting DRAPP tiles
drapp_aoi_gdf = gpd.sjoin(drapp_gdf, aoi_gdf, how='inner', predicate='intersects')

In [8]:
# Show AOI, bounding box, and DRAPP tiles
fig = plot_aoi_bbox_drapp(aoi_gdf, bbox_aoi_gdf, drapp_aoi_gdf)
fig.show()

/Users/ed.chan/Documents/GitHub/treebeard/utils/crop.py:123: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
# Spatial DRAPP tiles
drapp_tilepaths = save_drapp_tiles(drapp_tiles_dir, drapp_tilenames)

In [ ]:
drapp_tilepaths

In [ ]:
# Open all the files
src_files_to_mosaic = [rasterio.open(path) for path in drapp_tilepaths]
mosaic, out_trans = merge(src_files_to_mosaic)
red = mosaic[0]
green = mosaic[1]
blue = mosaic[2]
nir = mosaic[3]
rgb = np.dstack((red, green, blue))
plt.imshow(rgb)
# photo_date = pd.to_datetime(
#         drapp_aoi_gdf['photo_date']
#     ).dt.date.unique()[0].strftime('%Y-%m-%d')
# tile_name = drapp_aoi_gdf['tile'].unique()[0]
# plt.title(f'RGB Image: DRAPP Tile {tile_name} ({photo_date})')
for src in src_files_to_mosaic:
    src.close()
plt.show()